# Geonames Scraping
This script takes the list of reviews as input and enriches them with the Geonames ID.

In [16]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import InvalidSessionIdException
import string
import json
import time
import re

options = Options()
options.add_argument('headless')
options.add_argument('--lang=it')

driver = webdriver.Chrome(options=options)

df = pd.read_json ("data/reviews.json",lines=True)
#n = df.shape[0]
#print(n)

In [17]:
n = df.shape[0]
geonames_id = np.zeros(n)
df['geonames_id'] = geonames_id


In [19]:
countries = df.Hometown.unique()
countries

geonames = []


for i in range(len(countries)):
    splittato = countries[i].split(' ')

    #for j in range(len(splittato)):
    if(len(splittato)==2):
        
        splittato.insert(1, '+')
        #print(splittato)
        
    elif(len(splittato)==3):
        splittato.insert(1, '+')
        splittato.insert(3, '+')
        #print(splittato)
    
    api_geo = "".join(splittato)
    #print(api_geo)
    driver.get("https://www.geonames.org/search.html?q=" + api_geo)
    record = driver.find_element_by_xpath("//tbody/tr[3]/td[2]/a[1]").click() #click del primo link che trovo
    time.sleep(2)
    geonames_id = driver.find_element_by_xpath("//div[@data-geonameid]/div[2]/a").text #numero di geoname_id
    #print(api_geo + ':' + geonameid)
    new_geo = api_geo.replace('+',' ')
    #print(geonames_id)
    geonames.append(geonames_id)
    
    data = {'country': list(countries), 'geonames_id': geonames}
df_geo = pd.DataFrame(data)
df_geo['country'].unique()
    
    
driver.close()

In [21]:
for i in range(0,len(df_geo)):
    m = df['Hometown'] == df_geo['country'][i]
    df.loc[m, 'geonames_id'] = df_geo['geonames_id'][i]
df.to_json('data/reviews_with_geo.json')
df

Regno Unito
2635167
Italia
3175395
Spagna
2510769
Germania
2921044
Stati Uniti
6255150
Argentina
3865483
Corea del Sud
1835841
Paesi Bassi
2750405
Australia
2077456
Austria
2782113
Nuova Zelanda
2186224
Canada
6251999
Finlandia
660013
Francia
3017382
Cina
1814991
Svezia
2661886
Svizzera
2658434
Romania
798549
Israele
294640
Messico
3996063
Giappone
1861060
Brasile
3469034
Egitto
357994


,Name,City,Rating,Review,Hometown,Date_of_stay,Trip_type,geonames_id
0,Delle Vittorie Luxury Rooms & Suites,Palermo,5,Excellent facilities in a spacious luxurious l...,Regno Unito,settembre 2019,,2635167
1,Bed and Breakfast Il Rifugio,Caltanissetta,5,location molto comoda...Ti permette di postegg...,Italia,settembre 2019,Ha viaggiato con amici,3175395
2,Delle Vittorie Luxury Rooms & Suites,Palermo,5,estuvimos alojados 4 noches y no puedo poner n...,Spagna,febbraio 2020,,2510769
3,Garibaldi R&B,Messina,5,Graziosissimo R&B con vista sullo stretto. Ott...,Italia,gennaio 2020,Ha viaggiato per affari,3175395
4,Bed and Breakfast Il Rifugio,Caltanissetta,5,Arrivato in questo B&B quasi X caso e cioè gra...,Italia,giugno 2019,Ha viaggiato da solo,3175395
...,...,...,...,...,...,...,...,...
195,Bed & Breakfast Giulia,Caltanissetta,4,Sono stato con la mia famiglia dal 29 dicembre...,Italia,dicembre 2017,Ha viaggiato con la famiglia,3175395
196,Delle Vittorie Luxury Rooms & Suites,Palermo,5,Ho soggiornato più volte presso la struttura e...,Italia,settembre 2019,Ha viaggiato per affari,3175395
197,Garibaldi R&B,Messina,4,"Ci sono stato più volte, con la famiglia e da ...",Italia,ottobre 2015,Ha viaggiato con la famiglia,3175395
198,Bed & Breakfast Giulia,Caltanissetta,3,"Premetto che sono stato ospite di amici, quind...",Italia,settembre 2017,,3175395
